In [1]:
import argparse
import csv
import os
import shutil
import subprocess
import tempfile
from os.path import join
from pathlib import Path

import nibabel as nib
import numpy as np
from scipy.stats import ttest_1samp
from statsmodels.stats.multitest import multipletests
import scipy.ndimage as ndimage
from nilearn import datasets, image, plotting

In [4]:
data = np.load('/mnt/TeamShare/Data_Masterfile/Zahra-Thesis-Data/Msc_Proj_Nov/GLMOutputs-sub09-ses1/TYPED_FITHRF_GLMDENOISE_RR_sub09_ses01.npy', allow_pickle=True).item()
beta = data['betasmd']
beta.shape

(939701, 1, 1, 180)

In [6]:
beta = beta[:, 0, 0, 0:90]

IndexError: too many indices for array: array is 2-dimensional, but 4 were indexed

In [10]:
np.nanmax(beta)

np.float32(10774.517)

In [4]:
def _save_beta_overlay(data, anat_img, out_html, threshold_pct, vmax_pct, cut_coords, snapshot_path):
    finite = data[np.isfinite(data)]
    thr = float(np.percentile(finite, threshold_pct))
    vmax = float(np.percentile(finite, vmax_pct))
    img = nib.Nifti1Image(data, anat_img.affine, anat_img.header)
    view = plotting.view_img(img, bg_img=anat_img, cmap='inferno', symmetric_cmap=False, threshold=thr, vmax=vmax, colorbar=True,
        title=f'Mean |beta| (thr p{threshold_pct}={thr:.2f}, vmax p{vmax_pct}={vmax:.2f})', cut_coords=cut_coords)
    view.save_as_html(out_html)
    
    display = plotting.plot_stat_map(img, bg_img=anat_img, cmap='inferno', symmetric_cmap=False, threshold=thr, vmax=vmax, colorbar=True,
                                     title=f'Mean |beta| (thr p{threshold_pct}, vmax p{vmax_pct})', cut_coords=cut_coords)
    display.savefig(snapshot_path)
    display.close()

In [ ]:
sub, ses, run = '09', '1', '1'

In [3]:
bold_img = nib.load('/home/zkavian/Thesis_code_Glm_Opt/GLMsingle/sub-pd009_ses-1_run-1_task-mv_bold_corrected_smoothed_reg.nii.gz')
bold_data = bold+img.get_fdata()
anat = nib.load('/home/zkavian/Thesis_code_Glm_Opt/GLMsingle/sub-pd009_ses-1_T1w_brain.nii.gz')
back_mask = nib.load('/home/zkavian/Thesis_code_Glm_Opt/GLMsingle/sub-pd009_ses-1_T1w_brain_mask.nii.gz')
csf_mask = nib.load('/home/zkavian/Thesis_code_Glm_Opt/GLMsingle/sub-pd009_ses-1_T1w_brain_pve_0.nii.gz')
gray_mask = nib.load('/home/zkavian/Thesis_code_Glm_Opt/GLMsingle/sub-pd009_ses-1_T1w_brain_pve_1.nii.gz')
go_times = np.loadtxt('/home/zkavian/Thesis_code_Glm_Opt/GLMsingle/PSPD009-ses-1-go-times.txt')

In [ ]:
cut_coords = tuple(args.cut_coords) if args.cut_coords else _precentral_cut_coords(data_root, sub, ses)
roi_tag = args.overlay_stat + ("_pos" if args.overlay_positive_only else "")

if cached_beta_path.exists() and not args.force:
    beta_volume_filter = np.load(cached_beta_path)
    mean_clean_active = _compute_beta_summary(
        beta_volume_filter,
        overlay_stat=args.overlay_stat,
        overlay_positive_only=args.overlay_positive_only,
    )
    _save_beta_overlay(
        mean_clean_active,
        anat_img=anat_img,
        out_html=str(output_dir / f'clean_active_beta_overlay_sub{sub}_ses{ses}_run{run}.html'),
        threshold_pct=args.overlay_threshold_pct,
        vmax_pct=args.overlay_vmax_pct,
        cut_coords=cut_coords,
        snapshot_path=str(args.snapshot_path) if args.snapshot_path else None)
    if not args.skip_roi_ranking:
        roi_rank_path = output_dir / f'roi_{roi_tag}_sub{sub}_ses{ses}_run{run}.csv'
        _rank_rois_by_beta(
            mean_clean_active,
            anat_img=anat_img,
            anat_path=data_paths['anat'],
            ref_img=roi_ref_img,
            out_path=roi_rank_path,
            atlas_threshold=args.roi_atlas_threshold,
            label_patterns=args.roi_label_patterns,
            assume_mni=args.roi_assume_mni,
            mni_template=args.roi_mni_template,
        )
    return